## Together AI + RAG

[Together AI](https://python.langchain.com/docs/integrations/llms/together) has a broad set of OSS LLMs via inference API.

See [here](https://docs.together.ai/docs/inference-models). We use `"mistralai/Mixtral-8x7B-Instruct-v0.1` for RAG on the Mixtral paper.

Download the paper:
https://arxiv.org/pdf/2401.04088.pdf

In [ ]:
# Check for required packages and install if missing
import sys
required_packages = ['pypdf', 'chromadb', 'tiktoken', 'openai', 'langchain-together', 'langchain_community', 'langchain_core', 'langchain_together']
installed_packages = [pkg for pkg in required_packages if pkg in sys.modules]
missing_packages = [pkg for pkg in required_packages if pkg not in installed_packages]

if missing_packages:
    print(f"Installing missing packages: {' '.join(missing_packages)}")
    !pip install --quiet {' '.join(missing_packages)}

# Import functions and classes after installation
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_together import Together, TogetherEmbeddings


In [ ]:
# Load, split, and add to vectorDB
def process_pdf(file_path: str, model_name: str):
    loader = PyPDFLoader(file_path)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    all_splits = text_splitter.split_documents(data)

    embeddings = TogetherEmbeddings(model=model_name)
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        collection_name="rag-chroma",
        embedding=embeddings,
    )
    retriever = vectorstore.as_retriever()
    return retriever

retriever = process_pdf('~/Desktop/mixtral.pdf', 'togethercomputer/m2-bert-80M-8k-retrieval')

In [ ]:
# RAG prompt and chain
def rag_chain(retriever, question):
    template = "Answer the question based only on the following context:\n{context}\n
Question: {question}"
    prompt = ChatPromptTemplate.from_template(template)
    llm = Together(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        temperature=0.0,
        max_tokens=2000,
        top_k=1,
    )
    chain = (
        RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain.invoke(question)


In [ ]:
question = 'What are the Architectural details of Mixtral?'
response = rag_chain(retriever, question)

In [ ]:
response